In [124]:
# Importación de los datos
import sqlite3
# Para importar las librerías de 'pandas' y 'numpy' es necesario ejecutar: "pip install pandas numpy matplotlib seaborn scikit-learn jupyter"
import pandas as pd
import numpy as np
import os
os.chdir("C:/Users/a21gebremedingl/Desktop/AD/Proyecto1_AD_Python")

## 1. CARGAR PEDIDOS (CSV)

In [125]:
# Cargar el fichero pedidos.csv
df_pedidos = pd.read_csv("sources/pedidos.csv", encoding="utf-16", sep="|") # Usando la codificación correcta: "utf-16"
print(df_pedidos)

      Id Pedido  id_cliente  id_producto  fecha_pedido  cantidad   \
0            1.0      1263.0        531.0    2025-02-28       7.0   
1            NaN         NaN        744.0    2023-11-18       8.0   
2            3.0      3060.0        990.0    2023-10-18      10.0   
3            4.0      1216.0        174.0    2024-05-31       3.0   
4            5.0      1558.0        916.0    2024-06-30       9.0   
...          ...         ...          ...           ...       ...   
39995    39996.0      4739.0        503.0    2025-07-15       3.0   
39996    39997.0      1020.0        351.0    2025-03-07       4.0   
39997    39998.0      4163.0        196.0    2025-01-26       6.0   
39998    39999.0      2882.0        521.0    2023-11-04       2.0   
39999    40000.0      2417.0        284.0            20       6.0   

      precio_unitario     total    metodo_pago  estado_pedido   \
0               356.14   2492.98        Tarjeta      Pendiente   
1              1569.27  12554.16  Trans

## 2. CARGAR CLIENTES (JSON)


In [126]:
# Cargar el fichero clientes.json
df_clientes = pd.read_json("sources/clientes.json")
print(df_clientes)

      id_cliente                nombre                        email  \
0              1           Leah Oliver       morrisgary@example.org   
1              2             Gary Yang       patrickorr@example.org   
2              3           Diane Black  shannonwilliams@example.org   
3              4       Stephanie Ochoa     thorntonmary@example.net   
4              5       Manuel Petersen           judy84@example.org   
...          ...                   ...                          ...   
4995        4996          Nathan Smith     petersonpaul@example.com   
4996        4997            Wendy Owen       susanrusso@example.net   
4997        4998  Christopher Thompson       nicholas14@example.com   
4998        4999          Cindy Rogers      whitejoshua@example.com   
4999        5000          Selena Mills       wolfsteven@example.org   

                    telefono  \
0      001-330-515-0970x7851   
1               971.920.5709   
2        (431)706-3322x22301   
3     001-651-540-3

## 3. CARGAR PRODUCTOS (SQLite)


In [127]:
# Cargar el fichero productos.db
conexion = sqlite3.connect("sources/productos.db")
with conexion:
    df_productos = pd.read_sql_query("SELECT * FROM productos", conexion)   #  Acceso desde pandas: Convierte la tabla SQL en DataFrame directamente
print(df_productos)

#conexion.close() -->  Cerrar la conexión a la base de datos. // En este caso como se usa 'with', se cierra automáticamente al salir del bloque

    id_producto nombre_producto    categoria precio_base descuento   stock  \
0           1.0          Home X       Libros      469.99       0.0   367.0   
1          None      Threat Pro      Deporte      644.21      20.0    98.0   
2             3        Reduce X         Ropa     1417.93      15.0   309.0   
3           4.0        Half Max      Deporte      579.69      20.0   442.0   
4           5.0     Control Pro      Deporte      373.93       0.0  -99999   
..          ...             ...          ...         ...       ...     ...   
995       996.0          Lose X       Libros      782.92      10.0    47.0   
996       997.0      Degree Pro  Electrónica     1810.45      15.0   469.0   
997       998.0   Describe Lite           Ho     1735.67       0.0   360.0   
998       999.0        Hope Max        Hogar        8.16      15.0   277.0   
999      1000.0   Standard Lite  Electrónica      800.65      20.0   137.0   

                           proveedor  fecha_alta  
0           

## 4. NORMALIZAR TIPOS DE CLAVE

In [128]:
# Convertimos los valores de las columnas que actuarán de clave a str para evitar conflictos
df_clientes['id_cliente'] = df_clientes['id_cliente'].astype(str)
df_productos['id_producto'] = df_productos['id_producto'].astype(str)

# Como la intencion es hacer un merge en 'df_pedidos' utilizando 'id_cliente' e 'id_producto', es importante asegurarse de que los nombres de las columnas coincidan exactamente
df_pedidos.columns = df_pedidos.columns.str.strip() # Eliminar espacios antes y después de cada nombre de columna de df_pedidos
df_pedidos['id_producto'] = df_pedidos['id_producto'].astype(str)
df_pedidos['id_cliente'] = df_pedidos['id_cliente'].astype(str)

## 5. UNIÓN DE DATASETS

In [129]:
# Unir los tres datasets en uno solo
df_global = df_pedidos.merge(df_clientes, on='id_cliente', how='left')
df_global = df_pedidos.merge(df_productos, on='id_producto', how='left')
print(df_global)

      Id Pedido id_cliente id_producto fecha_pedido cantidad precio_unitario  \
0           1.0     1263.0       531.0   2025-02-28      7.0          356.14   
1           NaN        nan       744.0   2023-11-18      8.0         1569.27   
2           3.0     3060.0       990.0   2023-10-18     10.0          707.74   
3           4.0     1216.0       174.0   2024-05-31      3.0         1281.67   
4           5.0     1558.0       916.0   2024-06-30      9.0         1208.76   
...         ...        ...         ...          ...      ...             ...   
48658   39996.0     4739.0       503.0   2025-07-15      3.0          543.26   
48659   39997.0     1020.0       351.0   2025-03-07      4.0          134.56   
48660   39998.0     4163.0       196.0   2025-01-26      6.0         1855.47   
48661   39999.0     2882.0       521.0   2023-11-04      2.0          939.56   
48662   40000.0     2417.0       284.0           20      6.0           594.5   

          total    metodo_pago estado_p

## Apartado 1: Información básica del dataset


In [130]:
# 1.1 Mostrar las primeras 5 filas del dataset global
print(f"\n== Mostrando las 5 primeras filas del dataset global ==\n{df_global.head(5)}")

# 1.2 Mostrar las dimensiones del dataset (filas y columnas)
print(f"\n== Mostrando las dimensiones del dataset (filas y columnas) ==\n{df_global.shape}")

# 1.3 Mostrar los nombres de todas las columnas
print(f"\n== Mostrando los nombres de todas las columnas ==\n{df_global.columns.tolist()}")

# 1.4 Mostrar información sobre los tipos de datos de las columnas
# Proceso de optimización y verificación de los tipos de datos para mejorar el rendimiento

# Columnas que deben ser enteros
cols_enteros = ["Id Pedido", "id_cliente", "id_producto","cantidad", "stock"]
for col in cols_enteros:
    # Reemplazar valores inválidos por 0
    df_global[col] = df_global[col].replace(["###ERROR###", "-", None], "0")
    # Convertir a float primero (asegura que todos los valores sean numéricos)
    df_global[col] = df_global[col].astype(float)
    # Reemplazar valores NaN por 0
    df_global[col] = df_global[col].fillna(0)
    # Convertir a entero
    df_global[col] = df_global[col].astype(int)

# Columnas que deben ser float
cols_float = ["precio_unitario", "precio_base", "total", "descuento"]
for col in cols_float:
    df_global[col] = df_global[col].replace(["###ERROR###", "-", None], "0")
    df_global[col] = df_global[col].astype(float)
    df_global[col] = df_global[col].fillna(0)

# Salida de tipos de datos
print(f"\n== Mostrando información sobre los tipos de datos de las columnas ==\n{df_global.dtypes}")


# 1.5 Mostrar las últimas 3 filas del dataset
print(f"\n== Mostrando las 3 últimas filas del dataset global ==\n{df_global.tail(3)}")



== Mostrando las 5 primeras filas del dataset global ==
  Id Pedido id_cliente id_producto fecha_pedido cantidad precio_unitario  \
0       1.0     1263.0       531.0   2025-02-28      7.0          356.14   
1       NaN        nan       744.0   2023-11-18      8.0         1569.27   
2       3.0     3060.0       990.0   2023-10-18     10.0          707.74   
3       4.0     1216.0       174.0   2024-05-31      3.0         1281.67   
4       5.0     1558.0       916.0   2024-06-30      9.0         1208.76   

      total    metodo_pago estado_pedido        País envío nombre_producto  \
0   2492.98        Tarjeta     Pendiente           Namibia    Business Pro   
1  12554.16  Transferencia     Pendiente  Christmas Island       Serve Max   
2    7077.4         PayPal     Pendiente          Tanzania     Economy Max   
3   3845.01   Criptomoneda     Pendiente          Bulgaria        Only Max   
4  10878.84         PayPal       Enviado           Romania           Own X   

     categoria pr

## Apartado 2: Tipos de datos y valores nulos


In [131]:
# 2.1 Mostrar información detallada del DataFrame (tipos, memoria, nulos)
print(f"\n== Mostrando información detallada del DataFrame (tipos, memoria, nulos) == \n{df_global.info}")

# 2.2 Contar valores nulos por columna
print(f"\n== Mostrando cantidad de valores nulos por columna:\n{df_global.isnull().sum()}")

# 2.3 Mostrar solo las columnas que tienen valores nulos
print(f"\n== Mostrando solo las columnas que tienen valores nulos ==\n{df_global.columns[df_global.isnull().any()]}")

# 2.4 Calcular el porcentaje de valores nulos por columna
porcentaje_nulos = df_global.isnull().mean() * 100
print(f"\n== Mostrando el porcentaje de valores nulos por columna ==\n{porcentaje_nulos.round(2)}")

# 2.5 Verificar si hay filas completamente vacías
filas_vacias = df_global.isin([None, 0, 'NaN', 'nan', '###ERROR###']).all(axis=1)   # Según cómo se planteó más arriba, el 0 se considera vacío
if filas_vacias.any():
    print(f"\n== Mostrando filas completamente vacías ==\n{filas_vacias}")
else:
    print('\n== No se pueden mostrar las filas compltamente vacías porque no existe ninguna ==')



== Mostrando información detallada del DataFrame (tipos, memoria, nulos) == 
<bound method DataFrame.info of        Id Pedido  id_cliente  id_producto fecha_pedido  cantidad  \
0              1        1263          531   2025-02-28         7   
1              0           0          744   2023-11-18         8   
2              3        3060          990   2023-10-18        10   
3              4        1216          174   2024-05-31         3   
4              5        1558          916   2024-06-30         9   
...          ...         ...          ...          ...       ...   
48658      39996        4739          503   2025-07-15         3   
48659      39997        1020          351   2025-03-07         4   
48660      39998        4163          196   2025-01-26         6   
48661      39999        2882          521   2023-11-04         2   
48662      40000        2417          284           20         6   

       precio_unitario     total    metodo_pago estado_pedido  \
0       

## Apartado 3: Estadísticas descriptivas


In [159]:
# 3.1 Estadísticas descriptivas de todas las columnas numéricas
# Como se modificó el tipo de datos de las columnas numéricas, ahora se pueden calcular las estadísticas descriptivas
print(f"\n== Estadísticas descriptivas de todas las columnas numéricas ==\n{df_global.describe()}")

# 3.2 Estadísticas descriptivas de todas las columnas (numéricas y categóricas)
print(f"\n== Estadísticas descriptivas de todas las columnas (numéricas y categóricas) ==\n{df_global.describe(include='all')}")

# 3.3 Calcular la media, mediana y moda de la columna 'total'
media = np.mean(df_global['total'])
mediana = np.median(df_global['total'])
# Con pandas sería --> media = df_global['total'].mean(); mediana = df_global['total'].median()
moda = df_global['total'].mode()[0]
print(f"\n== Mostrando operaciones con la columna 'total' ==\n·Media: {media}\n·Mediana: {mediana}\n·Moda: {moda}")

# 3.4 Calcular el rango (máximo - mínimo) de la columna 'cantidad'
rango = np.ptp(df_global['cantidad'])
print(f"\n== Mostrando rango de la columna 'cantidad' ==\n{rango}")
# Con pandas sería --> rango = df_global['cantidad'].max() - df_global['cantidad'].min()

# 3.5 Calcular la desviación estándar y la varianza de la columna 'total'
desv_estandar = np.std(df_global['total'])
varianza = np.var(df_global['total'])
print(f"\n== Desviación estándar y varianza de la columna 'total' ==\n·Desviación estándar: {desv_estandar.round(3)}\n·Varianza: {varianza.round(3)}")

# 3.6 Contar valores únicos en columnas categóricas
print("\n", df_global.select_dtypes(include="object").nunique())




== Estadísticas descriptivas de todas las columnas numéricas ==
          Id Pedido    id_cliente   id_producto     cantidad  precio_unitario  \
count  48663.000000  48663.000000  48663.000000  48663.00000     48663.000000   
mean   17310.359760    763.965703 -17268.009371  -1692.06358      -724.094883   
std    19472.257146  12886.260882  38313.037292  12918.04197     12978.256737   
min   -99999.000000 -99999.000000 -99999.000000 -99999.00000    -99999.000000   
25%     8470.500000   1045.000000     41.000000      3.00000       412.830000   
50%    18801.000000   2320.000000    357.000000      5.00000       947.450000   
75%    29472.500000   3668.000000    681.000000      8.00000      1485.985000   
max    40000.000000   5000.000000   1000.000000     10.00000      2000.000000   

              total   precio_base     descuento         stock  
count  48663.000000  48663.000000  48663.000000  48663.000000  
mean    3633.652808   -356.266627  -1514.796704  -1158.405914  
std    14066.

## Apartado 4: Filtrado simple

In [133]:
# 4.1 Filtrar pedidos con cantidad mayor a 5 unidades


# 4.2 Filtrar pedidos con total mayor a 1000 euros


# 4.3 Filtrar pedidos con precio unitario menor a 50 euros


# 4.4 Filtrar pedidos con cantidad igual a 1


# 4.5 Mostrar los primeros 5 pedidos grandes (cantidad > 5)


# 4.6 Mostrar los primeros 5 pedidos caros (total > 1000€)



## Apartado 5: Filtrado con múltiples condiciones

In [134]:
# 5.1 Filtrar pedidos enviados con cantidad mayor a 3


# 5.2 Filtrar pedidos de tarjeta o PayPal con total mayor a 500 euros


# 5.3 Filtrar pedidos pendientes o cancelados con cantidad menor a 2


# 5.4 Filtrar pedidos enviados o devueltos con precio unitario mayor a 100


# 5.5 Mostrar los primeros 5 pedidos enviados grandes


# 5.6 Mostrar los primeros 5 pedidos premium



## Apartado 6: Filtrado temporal

In [135]:

# 6.1 Convertir la columna fecha_pedido a tipo datetime


# 6.2 Filtrar pedidos del año 2024


# 6.3 Filtrar pedidos del mes de enero de 2024


# 6.4 Filtrar pedidos del primer trimestre de 2024


# 6.5 Filtrar pedidos de los lunes (día de la semana = 0)


# 6.6 Mostrar los primeros 5 pedidos de 2024



## Apartado 7: Estado del pedido

In [136]:
# 7.1 Contar el número de pedidos por cada estado


# 7.2 Calcular el porcentaje de pedidos por estado


# 7.3 Calcular el total de ventas por estado


# 7.4 Calcular el valor promedio de pedidos por estado


# 7.5 Calcular la cantidad total de productos vendidos por estado


# 7.6 Mostrar estadísticas completas por estado



## Apartado 8: Método de pago

In [137]:
# 8.1 Contar el número de pedidos por método de pago


# 8.2 Calcular el porcentaje de uso de cada método de pago


# 8.3 Calcular el valor total de ventas por método de pago


# 8.4 Calcular el valor promedio de pedidos por método de pago


# 8.5 Calcular la cantidad promedio de productos por método de pago


# 8.6 Mostrar estadísticas completas por método de pago (numero de pedidos, total de ventas, promedio de pedidos, cantidad total, promedio de la cantidad)



## Apartado 9: País de envío

In [138]:
# 9.1 Contar el número total de países únicos


# 9.2 Mostrar los top 10 países por número de pedidos


# 9.3 Mostrar los top 10 países por valor total de ventas


# 9.4 Calcular el valor promedio de pedidos por país (top 10)


# 9.5 Calcular la cantidad total de productos enviados por país (top 10)


# 9.6 Mostrar estadísticas del país con más pedidos (número de pedidos, total de ventas, promedio de pedido y total de cantidad)





## Apartado 10: Análisis demográfico

In [139]:
# 10.1 Contar el número total de clientes únicos


# 10.2 Distribución de clientes por género


# 10.3 Porcentaje de distribución por género


# 10.4 Distribución de clientes por nivel de fidelización


# 10.5 Estadísticas básicas de edad


# 10.6 Mostrar estadísticas completas de edad por género (número de clientes, edad promedio, edad mínima, edad máxima y mediana de edad)




## Apartado 11: Clientes por país

In [140]:
# 11.1 Contar el número total de países únicos donde viven los clientes


# 11.2 Mostrar los top 15 países con más clientes


# 11.3 Calcular el porcentaje de clientes por país (top 10)


# 11.4 Mostrar estadísticas (número de clientes, edad promedio, edad mínima, edad máxima y mediana de edad) de edad por país (top 5 países)


# 11.5 Mostrar distribución de género por país (top 3 países)


# 11.6 Mostrar distribución de nivel de fidelización por país (top 3 países)




## Apartado 12: Clientes por ciudad

In [141]:
# 12.1 Mostrar las top 20 ciudades con más clientes


# 12.2 Mostrar las ciudades con exactamente 1 cliente


# 12.3 Mostrar las ciudades con más de 10 clientes




## Apartado 13: Categoría de productos

In [142]:
# 13.1 Mostrar las categorías cuyo nombre tiene más de 6 caracteres y que tienen entre 50 y 160 productos


# 13.2 Mostrar las categorías que contienen la letra "o" (mayúscula o minúscula) y tienen menos de 120 productos



## Apartado 14: Precios

In [143]:
# 14.1 Mostrar estadísticas básicas de precios base (precio promedio, precio mínimo, precio máximo y mediana)


# 14.2 Calcular el precio promedio por categoría


# 14.3 Encontrar los productos más caros y más baratos


# 14.4 Mostrar los top 10 productos más caros




## Apartado 15: Productos más vendidos

In [144]:

# 15.1 Obtener el producto más vendido (cantidad vendida, total vendido, nombre del producto y categoría)



## Apartado 16: Análisis temporal


In [145]:
# 16.1 Obtener información sobre las ventas por mes (número total de ventas, número de pedidos y cantidad total)


# 16.2 Obten la misma información pero en lugar de por mes, por semana.



## Apartado 17: Duplicados

In [146]:
## 17.1 Contar filas duplicadas en cada uno de los datasets originales.



## Apartado 18: Valores faltantes

In [147]:
# 18.1 Obtener el porcentaje de valores faltantes en cada columna del dataset.


## Apartado 19: Valoración final

Tras completar este ejercicio, ¿qué conclusiones has obtenido acerca de los datos? ¿Consideras que sería necesario aplicar algún tipo de preprocesamiento o crees que los datos son adecuados tal como están? Apoya tu valoración con ejemplos concretos de columnas que ilustren tu análisis.

<!-- Responde aquí al apartado 19 -->
**Responder**

